<a href="https://colab.research.google.com/github/sailesh2710/Vision-based-Obstacle-Detection-and-Navigation/blob/main/Moving_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# Install necessary dependencies
import os
import cv2
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt
from PIL import Image
import urllib.request
import tarfile

In [4]:
# Remove previous installations (Cleans up any conflicts)
!rm -rf models object_detection

# Clone the TensorFlow Model Garden repository again
!git clone --depth 1 https://github.com/tensorflow/models.git

# Install protobuf and dependencies
!apt-get -qq install libprotobuf-java protobuf-compiler

# Navigate to the research directory and compile all protobufs properly
!cd models/research && protoc object_detection/protos/*.proto --python_out=.

# Set the PYTHONPATH to avoid import issues
import os
os.environ['PYTHONPATH'] += ':/content/models/research:/content/models/research/slim'

# Copy the object detection directory for easy access
!cp -R models/research/object_detection/ object_detection/
!rm -rf models

Cloning into 'models'...
remote: Enumerating objects: 4321, done.
remote: Counting objects: 100% (4321/4321), done.
remote: Compressing objects: 100% (3339/3339), done.
remote: Total 4321 (delta 1208), reused 2041 (delta 909), pack-reused 0 (from 0)
Receiving objects: 100% (4321/4321), 53.31 MiB | 26.40 MiB/s, done.
Resolving deltas: 100% (1208/1208), done.


In [5]:
# Downgrade protobuf to a compatible version
!pip install --no-cache-dir protobuf==3.20.*

In [19]:
import tensorflow as tf
from google.protobuf import __version__ as protobuf_version

print(f"TensorFlow Version: {tf.__version__}")
print(f"Protobuf Version: {protobuf_version}")

TensorFlow Version: 2.18.0
Protobuf Version: 3.20.3


In [21]:
import tensorflow as tf

# Ensure Object Detection API imports work
try:
    from object_detection.utils import label_map_util
    from object_detection.utils import visualization_utils as vis_util
    print("Object Detection API successfully imported!")
except ImportError as e:
    print(f"Error importing Object Detection API: {e}")
    print("Try restarting runtime and running the setup again.")


Object Detection API successfully imported!


In [13]:
# Function to upload video from local machine
def upload_files():
    from google.colab import files
    uploaded = files.upload()
    for k, v in uploaded.items():
        open(k, 'wb').write(v)
    return list(uploaded.keys())

# Upload video
uploaded_files = upload_files()
video_path = uploaded_files[0] if uploaded_files else None
print(f"Video uploaded: {video_path}")

Saving Bike_Lane_Hero_New_Years_Day.mp4 to Bike_Lane_Hero_New_Years_Day.mp4
Video uploaded: Bike_Lane_Hero_New_Years_Day.mp4


In [15]:
# Define model name
MODEL_NAME = 'ssd_inception_v2_coco_2017_11_17'  # Using SSD Inception v2 for faster detection
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model used for object detection
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('object_detection/data', 'mscoco_label_map.pbtxt')
NUM_CLASSES = 10

# Download and extract the model
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())

tar_file.close()

# Create the detection graph
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

# Load label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


In [16]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

# Create a TensorFlow session
sess = tf.compat.v1.Session(graph=detection_graph)

# Define input and output tensors for object detection
# Input tensor: image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors: detection boxes, scores, and classes
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
num_detections = detection_graph.get_tensor_by_name('num_detections:0')


In [17]:
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)

    return masked_image

In [22]:
import cv2
import numpy as np
import time

# Open video file
video = cv2.VideoCapture("Bike_Lane_Hero_New_Years_Day.mp4")
fourcc = cv2.VideoWriter_fourcc(*'XVID')
frame_width = int(video.get(3))
frame_height = int(video.get(4))

# Define output video
out = cv2.VideoWriter('/content/drive/My Drive/Bike_Lane_Hero_New_Years_Day1.avi',
                      cv2.VideoWriter_fourcc('M','J','P','G'), 90, (frame_width, frame_height))

try:
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break

        # Start time for processing
        stime = time.time()
        frame_width = frame.shape[1]
        frame_height = frame.shape[0]
        rows, cols = frame.shape[:2]

        # Define movement boundaries
        left_boundary = [int(cols * 0.40), int(rows * 0.95)]
        left_boundary_top = [int(cols * 0.40), int(rows * 0.20)]
        right_boundary = [int(cols * 0.60), int(rows * 0.95)]
        right_boundary_top = [int(cols * 0.60), int(rows * 0.20)]
        bottom_left = [int(cols * 0.20), int(rows * 0.95)]
        top_left = [int(cols * 0.20), int(rows * 0.20)]
        bottom_right = [int(cols * 0.80), int(rows * 0.95)]
        top_right = [int(cols * 0.80), int(rows * 0.20)]
        vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)

        # Draw boundary lines
        cv2.polylines(frame, [vertices], isClosed=True, color=(255, 0, 0), thickness=5)

        # Apply region of interest mask
        copied = np.copy(frame)
        interested = region_of_interest(copied, vertices)
        frame_expanded = np.expand_dims(interested, axis=0)

        # Run object detection model
        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: frame_expanded})

        # Visualize detected objects
        vis_util.visualize_boxes_and_labels_on_image_array(
            frame,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8,
            min_score_thresh=0.78)

        # Extract bounding box coordinates
        ymin = int(boxes[0][0][0] * frame_height)
        xmin = int(boxes[0][0][1] * frame_width)
        ymax = int(boxes[0][0][2] * frame_height)
        xmax = int(boxes[0][0][3] * frame_width)
        print(f"Bounding Box Coordinates: ymin={ymin}, ymax={ymax}, xmin={xmin}, xmax={xmax}")
        print(f"Frame Size: {frame_height}x{frame_width}")
        print(f"Detection Confidence: {scores.max()}")

        # Overlay bounding box details
        cv2.putText(frame, f'y min = {ymin}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        cv2.putText(frame, f'y max = {ymax}', (50, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        cv2.putText(frame, f'x min = {xmin}', (50, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        cv2.putText(frame, f'x max = {xmax}', (50, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        # Movement decision
        if scores.max() > 0.78:
            print("inif")
            if xmin >= left_boundary[0]:
                print("Move LEFT - 1st !!!")
                cv2.putText(frame, 'Move LEFT!', (300, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
            elif xmax <= right_boundary[0]:
                print("Move RIGHT - 2nd !!!")
                cv2.putText(frame, 'Move RIGHT!', (300, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
            elif xmin <= left_boundary[0] and xmax >= right_boundary[0]:
                print("STOPPPPPP !!!! - 3rd !!!")
                cv2.putText(frame, 'STOPPPPPP!!!', (300, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)

        # Draw movement boundary lines
        cv2.line(frame, tuple(left_boundary), tuple(left_boundary_top), (255, 0, 0), 5)
        cv2.line(frame, tuple(right_boundary), tuple(right_boundary_top), (255, 0, 0), 5)

        # Write processed frame to video output
        out.write(frame)

except Exception as e:
    print(f"Error occurred: {e}")

finally:
    # Release resources
    video.release()
    out.release()
    cv2.destroyAllWindows()


Streaming output truncated to the last 5000 lines.
Detection Confidence: 0.9672375917434692
inif
Move RIGHT - 2nd !!!
Bounding Box Coordinates: ymin=242, ymax=667, xmin=471, xmax=650
Frame Size: 720x1280
Detection Confidence: 0.9757124781608582
inif
Move RIGHT - 2nd !!!
Bounding Box Coordinates: ymin=268, ymax=665, xmin=483, xmax=664
Frame Size: 720x1280
Detection Confidence: 0.9783076047897339
inif
Move RIGHT - 2nd !!!
Bounding Box Coordinates: ymin=263, ymax=664, xmin=491, xmax=666
Frame Size: 720x1280
Detection Confidence: 0.9778042435646057
inif
Move RIGHT - 2nd !!!
Bounding Box Coordinates: ymin=263, ymax=665, xmin=492, xmax=659
Frame Size: 720x1280
Detection Confidence: 0.977178692817688
inif
Move RIGHT - 2nd !!!
Bounding Box Coordinates: ymin=267, ymax=668, xmin=491, xmax=659
Frame Size: 720x1280
Detection Confidence: 0.9787629246711731
inif
Move RIGHT - 2nd !!!
Bounding Box Coordinates: ymin=259, ymax=669, xmin=486, xmax=661
Frame Size: 720x1280
Detection Confidence: 0.97746992

In [23]:
output_video = "/content/drive/My Drive/Bike_Lane_Hero_New_Years_Day1.avi"
if os.path.exists(output_video):
    print(f" Output video is saved at: {output_video}")
else:
    print("Output video not found. Check if VideoWriter saved it correctly.")

 Output video is saved at: /content/drive/My Drive/Bike_Lane_Hero_New_Years_Day1.avi
